# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:

* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM ERD

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database `'data.sqlite'`.

In [17]:
# Your code here
import pandas as pd 
import sqlite3 
conn = sqlite3.connect("data.sqlite.db")
cur = conn.cursor()


In [20]:
# ...existing code...
import pandas as pd
import sqlite3

db_path = r"c:\Users\DAMA\downloads\dsc-join-statements-lab\data.sqlite"
conn = sqlite3.connect(db_path)
cur = conn.cursor()

# quick test: list tables
pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;", conn)

# remember to close when finished
# conn.close()
# ...existing code...

,name
0,customers
1,employees
2,offices
3,orderdetails
4,orders
5,payments
6,productlines
7,products


## Select the names of all employees in Boston 

Hint: join the employees and offices tables. Select the first and last name.

In [ ]:
# Your code here
import pandas as pd
import sqlite3

db_path = r"c:\Users\DAMA\downloads\dsc-join-statements-lab\data.sqlite"
conn = sqlite3.connect(db_path)

query = """
SELECT e.firstName, e.lastName
FROM employees e
JOIN offices o ON e.officeCode = o.officeCode
WHERE o.city = 'Boston';
"""

df = pd.read_sql_query(query, conn)
print(df)



  firstName   lastName
0     Julie   Firrelli
1     Steve  Patterson


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by. Select the office code, city, and number of employees.

In [24]:
# Your code here
db_path = r"c:\Users\DAMA\downloads\dsc-join-statements-lab\data.sqlite"
conn = sqlite3.connect(db_path)

query = """
SELECT o.officeCode, o.city,
       COUNT(e.employeeNumber) AS num_employees
FROM offices o
LEFT JOIN employees e ON o.officeCode = e.officeCode
GROUP BY o.officeCode, o.city
HAVING COUNT(e.employeeNumber) = 0;
"""

df = pd.read_sql_query(query, conn)
print(df)


   officeCode    city  num_employees
0          27  Boston              0


## Write 3 questions of your own and answer them

In [28]:
# Answers will vary

# Example question: 
"""
How many customers are there per office?
""" 

db_path = r"c:\Users\DAMA\downloads\dsc-join-statements-lab\data.sqlite"
conn = sqlite3.connect(db_path)

query = """
SELECT o.officeCode,
       o.city,
       COUNT(c.customerNumber) AS num_customers
FROM offices o
LEFT JOIN employees e ON o.officeCode = e.officeCode
LEFT JOIN customers c ON e.employeeNumber = c.salesRepEmployeeNumber
GROUP BY o.officeCode, o.city
ORDER BY num_customers DESC;
"""

df = pd.read_sql_query(query, conn)
print(df)

# ...existing code...

   officeCode           city  num_customers
0           4          Paris             29
1           7         London             17
2           3            NYC             15
3           1  San Francisco             12
4           2         Boston             12
5           6         Sydney             10
6           5          Tokyo              5
7          27         Boston              0


In [33]:
"""
Question 1
Which employees have the most customers? show employee id, first/last name, and count of customers(top 10)
"""

# Your code here
db_path = r"c:\Users\DAMA\downloads\dsc-join-statements-lab\data.sqlite"
conn = sqlite3.connect(db_path)

query = """
SELECT e.employeeNumber,
       e.firstName,
       e.lastName,
       COUNT(c.customerNumber) AS num_customers
FROM employees e
LEFT JOIN customers c ON e.employeeNumber = c.salesRepEmployeeNumber
GROUP BY e.employeeNumber, e.firstName, e.lastName
ORDER BY num_customers DESC
LIMIT 10;
"""

df = pd.read_sql_query(query, conn)
print(df)

   employeeNumber firstName   lastName  num_customers
0            1401    Pamela   Castillo             10
1            1504     Barry      Jones              9
2            1323    George     Vanauf              8
3            1501     Larry       Bott              8
4            1286  Foon Yue      Tseng              7
5            1370    Gerard  Hernandez              7
6            1165    Leslie   Jennings              6
7            1166    Leslie   Thompson              6
8            1188     Julie   Firrelli              6
9            1216     Steve  Patterson              6


In [34]:
"""
Question 2
Top 5 products by total quantity sold.
"""

# Your code here
db_path = r"c:\Users\DAMA\downloads\dsc-join-statements-lab\data.sqlite"
conn = sqlite3.connect(db_path)

query = """
SELECT p.productCode,
       p.productName,
       SUM(od.quantityOrdered) AS total_quantity_sold
FROM products p
LEFT JOIN orderdetails od ON p.productCode = od.productCode
GROUP BY p.productCode, p.productName
ORDER BY total_quantity_sold DESC
LIMIT 5;
"""

df = pd.read_sql_query(query, conn)
print(df)

  productCode                              productName  total_quantity_sold
0    S18_3232              1992 Ferrari 360 Spider red                 1808
1    S18_1342                     1937 Lincoln Berline                 1111
2   S700_4002                American Airlines: MD-11S                 1085
3    S18_3856  1941 Chevrolet Special Deluxe Cabriolet                 1076
4    S50_1341             1930 Buick Marquette Phaeton                 1074


In [35]:
"""
Question 3
Which customers have spent the most (total order value)? Show top 10.
"""

# Your code here

db_path = r"c:\Users\DAMA\downloads\dsc-join-statements-lab\data.sqlite"
conn = sqlite3.connect(db_path)

query = """
SELECT c.customerNumber,
       c.customerName,
       ROUND(SUM(od.quantityOrdered * od.priceEach), 2) AS total_spent
FROM customers c
JOIN orders o ON c.customerNumber = o.customerNumber
JOIN orderdetails od ON o.orderNumber = od.orderNumber
GROUP BY c.customerNumber, c.customerName
ORDER BY total_spent DESC
LIMIT 10;
"""

df = pd.read_sql_query(query, conn)
print(df)
# ...existing code...

   customerNumber                  customerName  total_spent
0             141        Euro+ Shopping Channel    820689.54
1             124  Mini Gifts Distributors Ltd.    591827.34
2             114    Australian Collectors, Co.    180585.07
3             151            Muscle Machine Inc    177913.95
4             119             La Rochelle Gifts    158573.12
5             148       Dragon Souveniers, Ltd.    156251.03
6             323    Down Under Souveniers, Inc    154622.08
7             131             Land of Toys Inc.    149085.15
8             187                AV Stores, Co.    148410.09
9             450     The Sharp Gifts Warehouse    143536.27


## Level Up 1: Display the names of every individual product that each employee has sold

Hint: You will need to use multiple `JOIN` clauses to connect all the way from employee names to product names.

In [36]:
# Your code here

db_path = r"c:\Users\DAMA\downloads\dsc-join-statements-lab\data.sqlite"
conn = sqlite3.connect(db_path)

query = """
SELECT DISTINCT
       e.employeeNumber,
       e.firstName,
       e.lastName,
       p.productName
FROM employees e
JOIN customers c ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN orders o ON c.customerNumber = o.customerNumber
JOIN orderdetails od ON o.orderNumber = od.orderNumber
JOIN products p ON od.productCode = p.productCode
ORDER BY e.lastName, e.firstName, p.productName;
"""

df = pd.read_sql_query(query, conn)
print(df)

      employeeNumber firstName lastName                productName
0               1337      Loui   Bondur      18th century schooner
1               1337      Loui   Bondur     1900s Vintage Bi-Plane
2               1337      Loui   Bondur    1900s Vintage Tri-Plane
3               1337      Loui   Bondur          1903 Ford Model A
4               1337      Loui   Bondur        1904 Buick Runabout
...              ...       ...      ...                        ...
1363            1323    George   Vanauf              The Mayflower
1364            1323    George   Vanauf             The Queen Mary
1365            1323    George   Vanauf      The Schooner Bluenose
1366            1323    George   Vanauf                The Titanic
1367            1323    George   Vanauf  The USS Constitution Ship

[1368 rows x 4 columns]


## Level Up 2: Display the number of products each employee has sold

Alphabetize the results by employee last name.

Hint: Use the `quantityOrdered` column from `orderDetails`. Also, think about how to group the data when some employees might have the same first or last name.

In [38]:
# Your code here

db_path = r"c:\Users\DAMA\downloads\dsc-join-statements-lab\data.sqlite"
conn = sqlite3.connect(db_path)

query = """
SELECT e.employeeNumber,
       e.firstName,
       e.lastName,
       COALESCE(SUM(od.quantityOrdered), 0) AS total_products_sold
FROM employees e
LEFT JOIN customers c ON e.employeeNumber = c.salesRepEmployeeNumber
LEFT JOIN orders o ON c.customerNumber = o.customerNumber
LEFT JOIN orderdetails od ON o.orderNumber = od.orderNumber
GROUP BY e.employeeNumber, e.firstName, e.lastName
ORDER BY e.lastName, e.firstName;
"""

df = pd.read_sql_query(query, conn)
print(df)


db_path = r"c:\Users\DAMA\downloads\dsc-join-statements-lab\data.sqlite"
conn = sqlite3.connect(db_path)


    employeeNumber firstName   lastName  total_products_sold
0             1102    Gerard     Bondur                    0
1             1337      Loui     Bondur                 6186
2             1501     Larry       Bott                 8205
3             1143   Anthony        Bow                    0
4             1401    Pamela   Castillo                 9290
5             1076      Jeff   Firrelli                    0
6             1188     Julie   Firrelli                 4227
7             1611      Andy     Fixter                 6246
8             1702    Martin     Gerard                 4180
9             1370    Gerard  Hernandez                14231
10            1165    Leslie   Jennings                11854
11            1504     Barry      Jones                 7486
12            1625   Yoshimi       Kato                    0
13            1619       Tom       King                    0
14            1612     Peter      Marsh                 6632
15            1002     D

## Level Up 3: Display the names employees who have sold more than 200 different products

Hint: this is different from the previous question because the quantity sold doesn't matter, only the number of different products

In [43]:
# Your code here

db_path = r"c:\Users\DAMA\downloads\dsc-join-statements-lab\data.sqlite"
conn = sqlite3.connect(db_path)

query = """
SELECT e.employeeNumber,
       e.firstName,
       e.lastName,
       COUNT(DISTINCT od.productCode) AS num_distinct_products
FROM employees e
JOIN customers c ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN orders o ON c.customerNumber = o.customerNumber
JOIN orderdetails od ON o.orderNumber = od.orderNumber
GROUP BY e.employeeNumber, e.firstName, e.lastName
HAVING COUNT(DISTINCT od.productCode) > 200
ORDER BY num_distinct_products DESC;
"""

df = pd.read_sql_query(query, conn)
print(df)

Empty DataFrame
Columns: [employeeNumber, firstName, lastName, num_distinct_products]
Index: []


No employee has sold >200 distinct products

## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!